# Dependencies Installation

In [1]:
%pip install pymupdf
%pip install pillow
%pip install pandas
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Size of Scale Declaration


In [2]:
# Define the scale for X and Y dimensions to adjust the image accordingly.
scaleX=108
scaleY=108

# Importing Required Libraries

In [3]:
# Import necessary libraries for image processing, PDF handling, and drawing operations.
from PIL import Image, ImageDraw
from io import BytesIO
import fitz
import pandas as pd

# Common Variables and Settings

In [4]:
# General settings and variables used throughout the script.
color = 'red'
width = 2
TARGET_DPI = 300
# Coordinates of bbox extracted and modified based on the DE-TRAIN.csv file.
datas = [
    {"title": "Basic Information___First Name", "coordinate": [100, 325, 650, 365]},
    {"title": "Basic Information___Last Name", "coordinate": [675, 325, 1300, 365]},
    {"title": "Income___One_A", "coordinate": [1400, 1015, 1600, 1049]},
    {"title": "Income___One_B", "coordinate": [1400, 1049, 1600, 1083]},
    {"title": "Income___One_C", "coordinate": [1400, 1083, 1600, 1117]},
    {"title": "Income___One_D", "coordinate": [1400, 1112, 1600, 1150]},
    {"title": "Income___One_E", "coordinate": [1400, 1150, 1600, 1184]},
    {"title": "Income___One_F", "coordinate": [1400, 1180, 1600, 1219]},
    {"title": "Income___One_G", "coordinate": [1400, 1217, 1600, 1250]},
    {"title": "Income___One_H", "coordinate": [1400, 1250, 1600, 1284]},
    {"title": "Income___One_Z", "coordinate": [1400, 1284, 1600, 1352]},
    {"title": "Income___Two_B", "coordinate": [1400, 1352, 1600, 1384]},
    {"title": "Income___Three_B", "coordinate": [1400, 1384, 1600, 1417]},
    {"title": "Income___Four_B", "coordinate": [1400, 1417, 1600, 1450]},
    {"title": "Income___Five_B", "coordinate": [1400, 1450, 1600, 1484]},
    {"title": "Income___Six_B", "coordinate": [1400, 1484, 1600, 1515]},
    {"title": "Income___Seven",   "coordinate": [1400, 1515, 1600, 1585]},
    {"title": "Income___Eight",   "coordinate": [1400, 1585, 1600, 1618]},
    {"title": "Income___Nine",   "coordinate": [1400, 1618, 1600, 1650]},
    {"title": "Income___Ten",  "coordinate": [1400, 1650, 1600, 1685]},
    {"title": "Income___Eleven",  "coordinate": [1400, 1680, 1600, 1719]},
    {"title": "Income___Twelve",  "coordinate": [1400, 1715, 1600, 1752]},
    {"title": "Income___Thirteen",  "coordinate": [1400, 1748, 1600, 1785]},
    {"title": "Income___Fourteen",  "coordinate": [1400, 1780, 1600, 1819]},
    {"title": "Income___Fifteen",  "coordinate": [1400, 1814, 1600, 1854]},
]

# Function to Draw Bounding Boxes

In [17]:
# Draw rectangles on the image based on the specified coordinates to highlight the relevant sections.
def draw_bounding_boxes(page, datas):
    blocks = page.get_text("dict")['blocks']
    text_blocks = [block for block in blocks if block['type'] == 0]

    mat = fitz.Matrix(TARGET_DPI / scaleX, TARGET_DPI / scaleY)
    pix_map = page.get_pixmap(matrix=mat)
    image = Image.open(BytesIO(pix_map.tobytes()))
    draw = ImageDraw.Draw(image)

    for data in datas:
        for block in text_blocks:
            for line in block['lines']:
                for span in line['spans']:
                    x0, y0, x1, y1 = data["coordinate"]
                    if x0 > x1:
                        x0, x1 = x1, x0
                    if y0 > y1:
                        y0, y1 = y1, y0
                draw.rectangle((x0, y0, x1, y1), fill=None, outline=color, width=width)

    image.show()

# Function to Extract Text Coordinates from PDF

In [18]:
# Extract the coordinates of specific text in the PDF.
def extract_text_from_coordinates(pdf_dir, target_text):
    doc = fitz.open(pdf_dir)
    page = doc[0]

    target_x0, target_y0, target_x1, target_y1 = 0, 0, 0, 0

    blocks = page.get_text("dict")['blocks']
    text_blocks = [block for block in blocks if block['type'] == 0]

    for block in text_blocks:
        for line in block['lines']:
            for span in line['spans']:
                if target_text in span['text']:
                    x0, y0, x1, y1 = [coord * TARGET_DPI / scaleX for coord in span['bbox']]
                    target_x0, target_y0, target_x1, target_y1 = x0, y0, x1, y1

    return [target_x0, target_y0, target_x1, target_y1]

# Function to Process a Single PDF File

In [19]:
# Extract data from a single PDF file and return it as a DataFrame.
def process_pdf_file(pdf_dir, file_name, datas):
    doc = fitz.open(pdf_dir)
    page = doc[0]

    blocks = page.get_text("dict")['blocks']
    text_blocks = [block for block in blocks if block['type'] == 0]

    df = pd.DataFrame(columns=['file_name', 'label', 'value', 'x0', 'y0', 'x2', 'y2'])

    for data in datas:
        target_text = None
        for block in text_blocks:
            for line in block['lines']:
                for span in line['spans']:
                    x0, y0, x1, y1 = [coord * TARGET_DPI / scaleX for coord in span['bbox']]
                    target_x0, target_y0, target_x1, target_y1 = data["coordinate"]
                    if target_x0 <= x0 <= target_x1 and target_y0 <= y0 <= target_y1 and \
                            target_x0 <= x1 <= target_x1 and target_y0 <= y1 <= target_y1:
                        target_text = span['text']

        if target_text is not None:
            coordinate = extract_text_from_coordinates(pdf_dir=pdf_dir, target_text=target_text)

            new_row = {"file_name": file_name, 'label': data["title"], "value": target_text,
                       'x0': coordinate[0], 'y0': coordinate[1], 'x2': coordinate[2], "y2": coordinate[3]}
            df.loc[len(df)] = new_row 

    return df

# Function to Process Multiple PDF Files in a Dataset

In [20]:
# Process all PDF files in a specified directory and save the extracted data to a CSV file.
def process_pdf_dataset(directory, file_prefix, output_csv, datas):
    df = pd.DataFrame(columns=['file_name', 'label', 'value', 'x0', 'y0', 'x2', 'y2'])
    i = 1
    while True:
        if i == 201:
            break
        file_name = f"{file_prefix}_{i}.pdf"
        pdf_dir = f"{directory}/{file_name}"
        try:
            new_df = process_pdf_file(pdf_dir, file_name, datas)
            df = pd.concat([df, new_df], ignore_index=True)
        except Exception as e:
            print(f"File not found: {file_name}. Skipping... Error: {e}")
        i += 1
    
    pd.set_option('display.max_rows', 200)
    df.to_csv(output_csv, index=False)
    return df

# Processing and Drawing Bounding Boxes for a Single File

In [21]:
# Example usage: Processing a single PDF file and displaying the image with bounding boxes.
pdf_dir = "./Data Extraction/Train Files/1040_file_2.pdf"
doc = fitz.open(pdf_dir)
page = doc[0]
draw_bounding_boxes(page, datas)

# Processing and Extracting Data from Train and Test Datasets

In [10]:
# Process all PDF files in the Train and Test datasets and save the results to CSV files.
train_df = process_pdf_dataset("./Data Extraction/Train Files", "1040_file", "./train.csv", datas)
test_df = process_pdf_dataset("./Data Extraction/Test Files", "1040_file", "./test.csv", datas)

C:\Users\prash\AppData\Local\Temp\ipykernel_22920\616376168.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)


File not found: 1040_file_10.pdf. Skipping... Error: no such file: './Data Extraction/Train Files/1040_file_10.pdf'
File not found: 1040_file_16.pdf. Skipping... Error: no such file: './Data Extraction/Train Files/1040_file_16.pdf'
File not found: 1040_file_17.pdf. Skipping... Error: no such file: './Data Extraction/Train Files/1040_file_17.pdf'
File not found: 1040_file_19.pdf. Skipping... Error: no such file: './Data Extraction/Train Files/1040_file_19.pdf'
File not found: 1040_file_31.pdf. Skipping... Error: no such file: './Data Extraction/Train Files/1040_file_31.pdf'
File not found: 1040_file_46.pdf. Skipping... Error: no such file: './Data Extraction/Train Files/1040_file_46.pdf'
File not found: 1040_file_56.pdf. Skipping... Error: no such file: './Data Extraction/Train Files/1040_file_56.pdf'
File not found: 1040_file_57.pdf. Skipping... Error: no such file: './Data Extraction/Train Files/1040_file_57.pdf'
File not found: 1040_file_61.pdf. Skipping... Error: no such file: './Da

C:\Users\prash\AppData\Local\Temp\ipykernel_22920\616376168.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)


File not found: 1040_file_11.pdf. Skipping... Error: no such file: './Data Extraction/Test Files/1040_file_11.pdf'
File not found: 1040_file_12.pdf. Skipping... Error: no such file: './Data Extraction/Test Files/1040_file_12.pdf'
File not found: 1040_file_13.pdf. Skipping... Error: no such file: './Data Extraction/Test Files/1040_file_13.pdf'
File not found: 1040_file_14.pdf. Skipping... Error: no such file: './Data Extraction/Test Files/1040_file_14.pdf'
File not found: 1040_file_15.pdf. Skipping... Error: no such file: './Data Extraction/Test Files/1040_file_15.pdf'
File not found: 1040_file_18.pdf. Skipping... Error: no such file: './Data Extraction/Test Files/1040_file_18.pdf'
File not found: 1040_file_20.pdf. Skipping... Error: no such file: './Data Extraction/Test Files/1040_file_20.pdf'
File not found: 1040_file_21.pdf. Skipping... Error: no such file: './Data Extraction/Test Files/1040_file_21.pdf'
File not found: 1040_file_22.pdf. Skipping... Error: no such file: './Data Extra

# Accuracy Checking

In [12]:
from sklearn.metrics import mean_absolute_error
import pandas as pd

# Load the two CSV files into DataFrames
df_true = pd.read_csv("./Data Extraction/DE-TRAIN.csv")
df_pred = pd.read_csv("train.csv")

In [13]:
# Identify missing rows in df_true compared to df_pred
missing_rows = df_true[~df_true.isin(df_pred)].dropna()

# Display the missing rows
print("Missing Rows in df_true compared to df_pred:")
print(missing_rows)

Missing Rows in df_true compared to df_pred:
Empty DataFrame
Columns: [file_name, label, value, x0, y0, x2, y2]
Index: []


In [14]:
# Get the common column names
common_columns = {'x0', 'y0', 'x2', 'y2'}

# Initialize a dictionary to store Mean Absolute Errors for each column
mae_dict = {}
# Initialize an empty DataFrame with the desired columns for accuracy calculation
accuracy_df = pd.DataFrame(columns=[f"{common_column}_error" for common_column in common_columns])

# Calculate MAE and accuracy for each common column
for common_column in common_columns:
    mae = mean_absolute_error(df_true[common_column], df_pred[common_column])
    mae_dict[common_column] = mae
    accuracy_df[f"{common_column}_error"] = abs((df_true[common_column] - df_pred[common_column]) / df_true[common_column])

# Print the accuracy for each column
for common_column in common_columns:
    mean_value = accuracy_df[f"{common_column}_error"].mean()
    accuracy = 1 - mean_value
    print(f"Accuracy of {common_column} = {accuracy * 100:.2f}%")

Accuracy of x2 = 88.71%
Accuracy of y2 = 99.28%
Accuracy of x0 = 95.70%
Accuracy of y0 = 99.44%


In [15]:
# Specify the fields for comparison
fields_to_compare = ['value', 'label']

# Create a new column indicating whether the values are equal for each field and calculate accuracy
for field in fields_to_compare:
    df_pred[f'{field}_equal'] = (df_pred[field] == df_true[field]).astype(int)
    # Calculate accuracy
    accuracy = df_pred[f'{field}_equal'].mean() * 100
    # Display accuracy
    print(f"Accuracy of field '{field}': {accuracy:.2f}%")

Accuracy of field 'value': 58.34%
Accuracy of field 'label': 100.00%
